# ตัวอย่างการหาอนุพันธ์อันดับที่ 1 บนรูปภาพด้วย Sobel operator

In [1]:
import numpy as np
import cv2
import sys

print ( "Python", sys.version )
print ( "Numpy", np.__version__ )
print ( "OpenCV", cv2.__version__ )

Python 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
Numpy 1.19.2
OpenCV 4.5.1


In [2]:
import matplotlib
matplotlib.use('tkAgg')
from matplotlib import pyplot as plt

โหลดรูปภาพที่จะใช้ทดสอบมาเป็นภาพ 1-channel greyscale

In [3]:
dir = './fish.jpg'     # './circle_20px.png'  
src_8u = cv2.imread( dir , cv2.IMREAD_GRAYSCALE )
print( src_8u.shape) 

(200, 267)


กำหนดขนาดของ kernel ที่จะใช้ในตัวแปร **ksize** (ขนาดเป็นเลขคี่ตั้งแต่ 1, 3, 5, 7, 9, ...) นักศึกษาสามารถลองเปลี่ยนค่า ksize เพื่อดูผลลัพธ์ที่เปลี่ยนไปได้

เนื่องจาก Sobel operator คือ กระบวนการที่รวมเอา 2 kernels ได้แก่ 
- kernel สำหรับทำ Gaussian smooth 
- kernel สำหรับหาอนุพันธ์อันดับ 1 

รวมเข้าไว้ในตัวเดียวกัน ค่า ksize ที่กำหนดในที่นี้ ก็คือ ขนาดของ kernel สำหรับการทำ Gaussian smooth ส่วนแรกนั่นเอง

In [4]:
ksize = 9
if ksize == 3:
    ksize = -1   # use Scharr filter instead of Sobel

คำนวณผลลัพธ์ของการทำอนุพันธ์อันดับ 1 (= Gradient) ในแนวแกน X และแนวแกน Y แยกกันอย่างละ 1 ภาพด้วย Sobel operator ทั้งนี้ผลลัพธ์ที่ได้จะไม่ใช่ภาพชนิด Unsigned 8-bit integer (0-255) ตามปกติ แต่ต้องใช้เป็นภาพชนิด 64-bit float เพื่อให้รองรับผลลัพธ์ที่อาจมีค่าติดลบและค่าทศนิยมจากการทำ Sobel ได้

In [5]:
# Compute the 1st derivative in both axes
# Not use cv2.CV_8U (np.uint8) as it doesn't support negative value
sobelX_64f = cv2.Sobel( src_8u,
                        cv2.CV_64F,
                        1,          # order of X derivative
                        0,          # order of Y derivative 
                        ksize )     # kernel size (optional)
sobelY_64f = cv2.Sobel( src_8u,
                        cv2.CV_64F,
                        0,          # order of X derivative
                        1,          # order of Y derivative 
                        ksize )     # kernel size (optional)

# Test printing some values from the images
print( 'src_8u:', src_8u.dtype , src_8u[ :5 , 50:65 ] )
print( 'sobelX_64f:', sobelX_64f.dtype , sobelX_64f[ :5 , 50:65 ] )
print( 'sobelY_64f:', sobelY_64f.dtype , sobelY_64f[ :5 , 50:65 ] )

src_8u: uint8 [[79 79 79 79 79 79 79 79 79 80 80 81 81 81 80]
 [79 79 79 79 79 79 79 79 79 79 80 80 80 80 79]
 [79 79 79 79 79 79 79 79 79 79 79 79 79 79 79]
 [79 79 79 79 79 79 80 80 79 79 79 79 79 78 79]
 [78 78 78 78 78 78 80 80 80 80 79 79 79 79 79]]
sobelX_64f: float64 [[ 0.  0.  0.  0.  0.  0.  0.  0.  2.  4.  4.  2.  0. -4. -2.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  3.  3.  1.  0. -3.  0.]
 [ 0.  0.  0.  0.  0.  1.  1. -1. -1.  1.  1.  0. -1. -1.  3.]
 [ 0.  0.  0.  0.  0.  4.  4. -2. -2. -1. -1.  0. -2.  0.  3.]
 [ 0.  0.  0.  0.  0.  7.  7. -1. -1. -3. -3.  0. -1.  0.  1.]]
sobelY_64f: float64 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1. -3. -5. -7. -8. -7. -4.]
 [ 0.  0.  0.  0.  0.  1.  3.  3.  1. -1. -3. -4. -5. -5. -3.]
 [-4. -4. -4. -4. -4. -2.  2.  4.  4.  3.  1.  0.  0.  0. -1.]
 [-4. -4. -4. -4. -4. -3. -1.  1.  3.  3.  1.  0.  1.  2.  1.]]


คำนวณ Magnitude of Gradient และ Orientation of Gradient โดยใช้ภาพผลลัพธ์ทั้ง 2 ภาพจาก Sobel มาคำนวณร่วมกัน

In [6]:
# Compute magnitude and orientation of gradient
mag_64f = cv2.magnitude( sobelX_64f, sobelY_64f )
orient_360 = cv2.phase( sobelX_64f, sobelY_64f, angleInDegrees=True )

# Test printing some values from the images
np.set_printoptions(precision=1)
print( 'mag_64f:', mag_64f.dtype , mag_64f[ :5 , 50:65 ] )
print( 'orient_360:', orient_360.dtype , orient_360[ :5 , 50:65 ] )

mag_64f: float64 [[0.  0.  0.  0.  0.  0.  0.  0.  2.  4.  4.  2.  0.  4.  2. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  1.4 4.2 5.8 7.1 8.  7.6 4. ]
 [0.  0.  0.  0.  0.  1.4 3.2 3.2 1.4 1.4 3.2 4.  5.1 5.1 4.2]
 [4.  4.  4.  4.  4.  4.5 4.5 4.5 4.5 3.2 1.4 0.  2.  0.  3.2]
 [4.  4.  4.  4.  4.  7.6 7.1 1.4 3.2 4.2 3.2 0.  1.4 2.  1.4]]
orient_360: float64 [[  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.  180.  180. ]
 [  0.    0.    0.    0.    0.    0.    0.    0.  315.  315.  301.  278.1
  270.  246.8 270. ]
 [  0.    0.    0.    0.    0.   45.   71.6 108.4 135.  315.  288.4 270.
  258.7 258.7 315. ]
 [270.  270.  270.  270.  270.  333.4  26.6 116.6 116.6 108.4 135.    0.
  180.    0.  341.6]
 [270.  270.  270.  270.  270.  336.8 351.9 135.  108.4 135.  161.6   0.
  135.   90.   45. ]]


หากลองแสดงผลภาพ sobelX_64f, sobelY_64f, mag_64f และ orient_360 ซึ่งเป็นภาพชนิด 64-bit float ออกมาด้วยคำสั่ง cv2.imshow จะเห็นว่าแสดงผลได้ แต่ภาพที่ได้อาจไม่ตรงกับปกติที่เราคุ้นกันว่า 0=ดำปี๋ และ 255=ขาวจั๊วะ

In [7]:
cv2.imshow('sobelX_64f',sobelX_64f)
cv2.imshow('sobelY_64f',sobelY_64f)
cv2.imshow('mag_64f',mag_64f)
cv2.imshow('orient_360',orient_360)
cv2.waitKey(1)

-1

เพื่อปรับสเกลทุกภาพให้เป็น Unsigned 8-bit integer (0-255) ตามที่เราคุ้นเคยกันดี จึงเพิ่มคำสั่งการ normalize รูปภาพทั้ง 4 เข้าไป โดยการ normalize รูปภาพจาก 64-bit float ไปเป็น Unsigned 8-bit integer ในที่นี้ใช้การใส่ absolute เพื่อให้ไม่มีค่าติดลบก่อน แล้วจึงนำค่ามาสเกลแบบบัญญัติไตรยางค์ธรรมดา (ทั้งนี้นักศึกษาสามารถเปลี่ยนไปใช้วิธีการ normalize วิธีอื่นได้ตามความเหมาะสม)

In [8]:
# Simply convert the input image to CV_8U (np.uint8) by neglecting negative sign
def convert_normalize_8u ( img ):
    img_abs = np.absolute ( img )
    img_8u = np.uint8 ( img_abs * ( 255 / img_abs.max() ) )
    return img_8u

# Convert to cv2.CV_8U
sobelX_8u = convert_normalize_8u ( sobelX_64f )
sobelY_8u = convert_normalize_8u ( sobelY_64f )
mag_8u = convert_normalize_8u ( mag_64f )
orient_8u = convert_normalize_8u ( orient_360 )

# Test printing some values from the images
print( 'sobelX_8u:', sobelX_8u.dtype , sobelX_8u[ :5 , 50:65 ] )
print( 'sobelY_8u:', sobelY_8u.dtype , sobelY_8u[ :5 , 50:65 ] )
print( 'mag_8u:', mag_8u.dtype , mag_8u[ :5 , 50:65 ] )
print( 'orient_8u:', orient_8u.dtype , orient_8u[ :5 , 50:65 ] )

sobelX_8u: uint8 [[0 0 0 0 0 0 0 0 0 1 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 2 2 0 0 0 0 0 0 0 0]]
sobelY_8u: uint8 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 2 2 2 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 0]
 [1 1 1 1 1 0 0 1 1 0 0 0 0 0 0]
 [1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]]
mag_8u: uint8 [[0 0 0 0 0 0 0 0 0 1 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 2 2 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0]
 [1 1 1 1 1 2 1 0 0 1 0 0 0 0 0]]
orient_8u: uint8 [[  0   0   0   0   0   0   0   0   0   0   0   0   0 127 127]
 [  0   0   0   0   0   0   0   0 223 223 213 197 191 175 191]
 [  0   0   0   0   0  31  50  76  95 223 204 191 183 183 223]
 [191 191 191 191 191 236  18  82  82  76  95   0 127   0 242]
 [191 191 191 191 191 238 249  95  76  95 114   0  95  63  31]]


แสดงผลลัพธ์ทั้งหมดรวมไว้ในหน้าต่างแยกของ matplotlib

In [9]:
# Put everything in lists
titles = ['Original',
          'Sobel X ('+str(ksize)+'x'+str(ksize)+')',
          'Sobel Y ('+str(ksize)+'x'+str(ksize)+')',
          'Magnitude','Orientation',
          '','','','','']
if ksize == -1:
    titles[1:3] = ['Scharr X (3x3)','Scharr Y (3x3)' ]
images = [src_8u, sobelX_8u, sobelY_8u, mag_8u, orient_8u,
          None,
          cv2.applyColorMap ( sobelX_8u, cv2.COLORMAP_JET ),
          cv2.applyColorMap ( sobelY_8u, cv2.COLORMAP_JET ),
          cv2.applyColorMap ( mag_8u, cv2.COLORMAP_JET ),
          cv2.applyColorMap ( orient_8u, cv2.COLORMAP_JET ) ]

# Plot all results using pyplot
plt.figure("Sobel operator: first-order derivative")
for i in range ( len(images) ):
    if images[i] is None:
        continue
    plt.subplot(2,5,i+1)
    plt.title(titles[i])
    if titles[i] != '':
        plt.imshow( images[i], cmap='gray' )
    else:
        plt.imshow( cv2.cvtColor( images[i], cv2.COLOR_BGR2RGB ) )
    plt.xticks( [] ) ; plt.yticks( [] )

plt.show()

In [10]:
cv2.destroyAllWindows()